In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import statsmodels as sm
from pathlib import Path
import yfinance as yf
from tqdm import tqdm

from correlation_helper import *

In [2]:
# Initialize workflow parameters
num_top = 50
data_path = Path(f'../data/top_{num_top}_companies_by_sector.csv').resolve()
market_ticker = '^GSPC'
analysis_start = "2024-01-01"
analysis_end = "2025-01-01"

In [3]:
# Import data
data = pd.read_csv(data_path)#.sample(10, replace=False)
data

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap (B),Country,IPO Year,Volume,Sector,Industry
0,UFPI,UFP Industries Inc. Common Stock,$115.67,-0.7900,-0.678%,7.024022,United States,1993.0,223820,Basic Materials,Forest Products
1,AMWD,American Woodmark Corporation Common Stock,$77.01,0.4800,0.627%,1.159322,United States,1986.0,168642,Basic Materials,Forest Products
2,IPX,IperionX Limited American Depositary Share,$26.52,0.1200,0.455%,0.682213,United States,NaN,50560,Basic Materials,Other Metals and Minerals
3,EU,enCore Energy Corp. Common Shares,$2.91,0.0600,2.105%,0.541518,United States,NaN,1091464,Basic Materials,Other Metals and Minerals
4,USGO,U.S. GoldMining Inc. Common stock,$12.59,1.7800,16.466%,0.156678,United States,2023.0,225247,Basic Materials,Precious Metals
...,...,...,...,...,...,...,...,...,...,...,...
524,VIVK,Vivakor Inc. Common Stock,$0.845,-0.0248,-2.851%,0.028424,United States,NaN,42549,Utilities,Environmental Services
525,SONM,Sonim Technologies Inc. Common Stock,$2.55,-0.0800,-3.042%,0.012423,United States,2019.0,15396,Utilities,Telecommunications Equipment
526,CLRO,ClearOne Inc. (DE) Common Stock,$0.4926,-0.0072,-1.441%,0.011807,United States,NaN,87213,Utilities,Telecommunications Equipment
527,SUNE,SUNation Energy Inc. Common Stock,$1.225,-0.0950,-7.197%,0.002223,United States,NaN,87057,Utilities,Telecommunications Equipment


In [4]:
# Download Stock Data
stock_data = yf.download(data['Symbol'].to_list(), start=analysis_start, end=analysis_end).dropna(axis=1, how='all')['Close']
stock_data.head()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  529 of 529 completed

20 Failed downloads:
['ADNWW', 'FOXXW', 'NEOVW', 'USGOW', 'HYMCW', 'RMCOW', 'AMODW', 'NIOBW', 'SLDPW', 'NESRW', 'ZEOWW', 'ANNAW', 'DFLIW', 'NXPLW', 'ARKOW', 'MVSTW', 'DHCNL', 'NEHCW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-01-01 -> 2025-01-01)')
['STRK', 'SFD']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-01-01 -> 2025-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1704085200, endDate = 1735707600")')


Ticker,AAL,AAON,AAPL,ABAT,ABNB,ACDC,ACTG,ADBE,ADI,ADN,...,XPON,YORW,Z,ZBRA,ZD,ZEO,ZG,ZION,ZM,ZS
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-02,13.44,73.634888,184.532089,4.405,134.479996,7.98,3.92,580.070007,189.526352,6.60,...,501.0,37.200623,57.250000,267.980011,67.309998,11.26,56.220001,42.250351,69.150002,212.369995
2024-01-03,12.95,72.847717,183.150375,4.230,133.419998,7.95,3.89,571.789978,185.003113,6.15,...,509.0,36.628456,55.230000,252.520004,65.860001,11.28,53.980000,40.201965,67.169998,210.240005
2024-01-04,13.09,73.475456,180.824356,4.250,133.720001,7.64,3.82,567.049988,182.173615,6.00,...,521.0,36.250240,54.160000,252.970001,65.570000,11.27,52.750000,40.814568,66.900002,210.330002
2024-01-05,13.60,72.190086,180.098694,4.150,135.979996,8.00,3.84,564.599976,182.643555,5.91,...,517.0,35.901115,53.709999,252.690002,65.160004,11.27,52.430000,42.164204,66.959999,209.809998
2024-01-08,14.58,73.435608,184.452560,4.120,140.080002,7.95,3.89,580.549988,185.022659,5.94,...,509.0,36.153263,55.669998,261.089996,66.650002,11.28,54.580002,42.633228,68.389999,218.100006


In [5]:
stock_data.columns

Index(['AAL', 'AAON', 'AAPL', 'ABAT', 'ABNB', 'ACDC', 'ACTG', 'ADBE', 'ADI',
       'ADN',
       ...
       'XPON', 'YORW', 'Z', 'ZBRA', 'ZD', 'ZEO', 'ZG', 'ZION', 'ZM', 'ZS'],
      dtype='object', name='Ticker', length=509)

In [6]:
grouped_columns = data.sort_values('Sector')['Symbol'].to_list()
grouped_columns = [stock for stock in grouped_columns if stock in stock_data.columns]
len(grouped_columns)
stock_data = stock_data[grouped_columns]


In [7]:
market_returns = standardize(pct_change(yf.download(market_ticker, start="2024-01-01", end="2025-02-01")['Close'].to_numpy().flatten()))
market_returns

[*********************100%***********************]  1 of 1 completed


array([-1.11628555e+00, -5.43501384e-01,  1.12446784e-01,  1.64666951e+00,
       -2.99992104e-01,  5.91890260e-01, -1.99260896e-01, -2.17263008e-02,
       -5.81324530e-01, -8.16728363e-01,  9.83817262e-01,  1.42180646e+00,
        1.58469811e-01,  2.49238820e-01, -1.41165985e-02,  5.41252016e-01,
       -1.96855077e-01,  8.27949042e-01, -1.90471048e-01, -2.12621673e+00,
        1.44430185e+00,  1.21842560e+00, -5.13283240e-01,  1.72962612e-01,
        9.13428835e-01, -4.42475228e-02,  6.01424687e-01, -2.33955292e-01,
       -1.82265559e+00,  1.08051061e+00,  6.11276163e-01, -7.15175389e-01,
       -8.65221564e-01,  4.23475924e-02,  2.52162058e+00, -7.20426529e-02,
       -5.88242374e-01,  9.75481274e-02, -3.22496104e-01,  5.37347985e-01,
        8.84261238e-01, -2.64456174e-01, -1.38804810e+00,  5.26362618e-01,
        1.17094448e+00, -9.30540503e-01, -2.55589129e-01,  1.28301355e+00,
       -3.55751856e-01, -4.73926111e-01, -9.24847346e-01,  6.73300649e-01,
        5.89790074e-01,  

In [8]:
# Pre-calculate all standardized returns
all_returns = pd.DataFrame(index=range(stock_data.shape[0]-1), columns=stock_data.columns)
for col in stock_data.columns:
    all_returns[col] = standardize(pct_change(stock_data[col].values))

# Initialize matrices
n_stocks = len(stock_data.columns)
pearson = pd.DataFrame(np.eye(n_stocks), columns=stock_data.columns, index=stock_data.columns)
beta = pd.DataFrame(np.eye(n_stocks), columns=stock_data.columns, index=stock_data.columns)
r_squared = pd.DataFrame(np.eye(n_stocks), columns=stock_data.columns, index=stock_data.columns)

# Only calculate upper triangle
for i in tqdm(range(n_stocks)):
    for j in range(i+1, n_stocks):
        # Get clean paired returns
        clean_returns1, clean_returns2 = remove_outliers(all_returns.iloc[:,i], all_returns.iloc[:,j])
        
        if clean_returns1.shape[0] < 230:
            pearson.iloc[i,j] = pearson.iloc[j,i] = np.nan
            beta.iloc[i,j] = beta.iloc[j,i] = np.nan
            r_squared.iloc[i,j] = r_squared.iloc[j,i] = np.nan
            continue
            
        # Calculate all metrics at once
        p_corr, b_corr, r2_corr = compute_correlation_metrics(
            clean_returns1, clean_returns2, market_returns, 60)
        
        # Fill both sides of symmetric matrices
        pearson.iloc[i,j] = pearson.iloc[j,i] = p_corr
        beta.iloc[i,j] = beta.iloc[j,i] = b_corr
        r_squared.iloc[i,j] = r_squared.iloc[j,i] = r2_corr

100%|██████████| 509/509 [10:09<00:00,  1.20s/it]


In [9]:
pearson.to_csv(f'../data/pearson_matrix_{analysis_start}_{analysis_end}_top{num_top}.csv')
beta.to_csv(f'../data/beta_matrix_{analysis_start}_{analysis_end}_top{num_top}.csv')
r_squared.to_csv(f'../data/r_squared_matrix_{analysis_start}_{analysis_end}_top{num_top}.csv')

In [10]:
pearson

Ticker,UFPI,HYMC,ABAT,HYMCL,USAU,USGO,EU,IPX,AMWD,NB,...,TLN,OTTR,NFE,NEXT,SJW,INFN,CDZIP,NNE,NWE,ALCE
Ticker,,,,,,,,,,,,,,,,,,,,,
UFPI,1.000000,0.276080,0.174083,0.085249,0.227197,0.177281,0.224328,0.022708,0.721342,0.138928,...,0.276280,0.537765,0.270071,0.267073,0.338355,0.302005,0.103602,NaN,0.451092,0.060193
HYMC,0.276080,1.000000,0.083090,-0.005893,0.395763,0.221370,0.253970,0.018191,0.190664,0.108390,...,0.089374,0.185726,0.178155,0.231937,0.133157,0.275507,0.056519,NaN,0.193117,0.121040
ABAT,0.174083,0.083090,1.000000,0.144244,0.102481,0.085672,0.075980,0.049578,0.202218,0.089839,...,-0.023538,0.077462,-0.023517,0.054093,0.062329,0.176765,0.020386,NaN,0.063528,0.033935
HYMCL,0.085249,-0.005893,0.144244,1.000000,-0.012104,0.111547,-0.002272,-0.087344,0.039080,-0.012179,...,-0.009708,-0.006485,-0.036845,0.218882,0.041197,0.075511,-0.066864,NaN,0.040052,-0.111810
USAU,0.227197,0.395763,0.102481,-0.012104,1.000000,0.197453,0.290732,0.088652,0.091158,0.078014,...,0.150170,0.105648,0.154083,0.125537,0.186197,0.206021,0.008318,NaN,0.185522,0.098932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INFN,0.302005,0.275507,0.176765,0.075511,0.206021,0.116514,0.345201,0.101086,0.229472,0.038991,...,0.138298,0.155662,0.107629,0.141406,0.093484,1.000000,-0.052503,NaN,0.149209,0.011942
CDZIP,0.103602,0.056519,0.020386,-0.066864,0.008318,-0.054313,-0.028409,-0.047706,0.095630,-0.051520,...,0.067164,0.162116,0.192285,0.136278,0.027864,-0.052503,1.000000,NaN,0.162962,0.055130
NNE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [11]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Create a figure with a larger size
# plt.figure(figsize=(120, 100))

# # Create heatmap
# sns.heatmap(pearson, 
#             annot=True,  # Show correlation values
#             cmap='RdBu',  # Red-Blue diverging colormap
#             vmin=-1, vmax=1,  # Fix scale from -1 to 1
#             center=0,  # Center colormap at 0
#             fmt='.2f',  # Show 2 decimal places
#             square=True,  # Make cells square
#             xticklabels=pearson.columns,  # Show x-axis labels
#             yticklabels=pearson.columns
#             )  # Show y-axis labels

# # Customize the plot
# plt.title('Pearson Correlation Heatmap of Stock Returns', pad=20)
# plt.xlabel('Stock Symbols')
# plt.ylabel('Stock Symbols')

# # Rotate x-axis labels for better readability
# plt.xticks(rotation=45, ha='right')
# plt.yticks(rotation=0)

# # Adjust layout to prevent label cutoff
# plt.tight_layout()

# # Show the plot
# plt.show()

In [12]:
beta

Ticker,UFPI,HYMC,ABAT,HYMCL,USAU,USGO,EU,IPX,AMWD,NB,...,TLN,OTTR,NFE,NEXT,SJW,INFN,CDZIP,NNE,NWE,ALCE
Ticker,,,,,,,,,,,,,,,,,,,,,
UFPI,1.000000,0.400641,0.069862,0.289755,0.746092,-0.464907,0.231779,0.041877,0.891955,0.575252,...,0.433951,0.520609,0.263500,0.552863,0.540858,0.704196,-0.497301,NaN,0.745554,-0.324495
HYMC,0.400641,1.000000,0.142288,0.183576,0.303395,0.602544,0.541550,-0.344446,0.328951,0.158555,...,0.247765,0.650708,0.681506,0.691727,0.168833,0.830058,-0.046983,NaN,-0.264505,-0.395644
ABAT,0.069862,0.142288,1.000000,-0.839743,-0.480638,-0.188233,0.668402,0.457811,0.175705,0.530757,...,0.337088,0.047634,0.170640,0.277047,0.698178,0.775765,0.189429,NaN,0.126654,-0.228034
HYMCL,0.289755,0.183576,-0.839743,1.000000,-0.586419,-0.257207,-0.183246,-0.755363,-0.071777,0.062766,...,-0.124098,0.023760,-0.350026,0.321602,0.030585,0.310486,0.076734,NaN,0.242429,0.374013
USAU,0.746092,0.303395,-0.480638,-0.586419,1.000000,0.559888,0.819386,0.010906,0.282621,0.549037,...,0.265188,0.373202,-0.129888,0.860991,0.076294,0.144757,0.042192,NaN,-0.177526,0.592777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INFN,0.704196,0.830058,0.775765,0.310486,0.144757,-0.379417,0.891028,0.564666,0.137195,0.745489,...,0.336555,-0.044225,0.149974,0.858507,0.640518,1.000000,0.654822,NaN,-0.076239,-0.044663
CDZIP,-0.497301,-0.046983,0.189429,0.076734,0.042192,0.369749,0.318166,-0.508290,0.172624,-0.086296,...,-0.443698,0.117140,-0.016626,0.011739,-0.182719,0.654822,1.000000,NaN,0.385010,0.581557
NNE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [16]:
# Create a figure with a larger size
plt.figure(figsize=(120, 100))

# Create heatmap
sns.heatmap(beta, 
            annot=True,  # Show correlation values
            cmap='RdBu',  # Red-Blue diverging colormap
            vmin=-1, vmax=1,  # Fix scale from -1 to 1
            center=0,  # Center colormap at 0
            fmt='.2f',  # Show 2 decimal places
            square=True,  # Make cells square
            xticklabels=pearson.columns,  # Show x-axis labels
            yticklabels=pearson.columns
            )  # Show y-axis labels

# Customize the plot
plt.title('Beta Correlation Heatmap of Stock Returns', pad=20)
plt.xlabel('Stock Symbols')
plt.ylabel('Stock Symbols')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

# Adjust layout to prevent label cutoff
plt.tight_layout()

# Show the plot
plt.show()

NameError: name 'plt' is not defined

In [14]:
# # Create a figure with a larger size
# plt.figure(figsize=(120, 100))

# # Create heatmap
# sns.heatmap(r_squared, 
#             annot=True,  # Show correlation values
#             cmap='RdBu',  # Red-Blue diverging colormap
#             vmin=0, vmax=0.5,  # Fix scale from -1 to 1
#             center=0,  # Center colormap at 0
#             fmt='.2f',  # Show 2 decimal places
#             square=True,  # Make cells square
#             xticklabels=pearson.columns,  # Show x-axis labels
#             yticklabels=pearson.columns
#             )  # Show y-axis labels

# # Customize the plot
# plt.title('R Squared Correlation Heatmap of Stock Returns', pad=20)
# plt.xlabel('Stock Symbols')
# plt.ylabel('Stock Symbols')

# # Rotate x-axis labels for better readability
# plt.xticks(rotation=45, ha='right')
# plt.yticks(rotation=0)

# # Adjust layout to prevent label cutoff
# plt.tight_layout()

# # Show the plot
# plt.show()

In [15]:
# # Get pairs with correlation between 0.7 and 0.95
# pairs_data = []
# for i in range(len(pearson.columns)):
#     for j in range(i+1, len(pearson.columns)):  # Start from i+1 to avoid duplicates and self-pairs
#         corr = pearson.iloc[i,j]
#         if 0.7 < corr < 0.95:
#             stock1 = pearson.columns[i]
#             stock2 = pearson.columns[j]
#             stock1_sector = data[data['Symbol'] == stock1]['Sector'].values[0]
#             stock2_sector = data[data['Symbol'] == stock2]['Sector'].values[0]
            
#             pairs_data.append({
#                 'Stock1': stock1,
#                 'Stock2': stock2,
#                 'Stock1_Name': data[data['Symbol'] == stock1]['Name'].values[0],
#                 'Stock2_Name': data[data['Symbol'] == stock2]['Name'].values[0],
#                 'Stock1_Sector': stock1_sector,
#                 'Stock2_Sector': stock2_sector,
#                 'Pearson_Correlation': corr,
#                 'Beta_Correlation': beta.iloc[i,j],
#                 'R_Squared': r_squared.iloc[i,j]
#             })

# # Create DataFrame and sort by correlation
# pairs_df = pd.DataFrame(pairs_data)
# pairs_df = pairs_df.sort_values('Pearson_Correlation', ascending=False)

# # Display the DataFrame
# print(f"\nFound {len(pairs_df)} pairs with correlation between 0.7 and 0.95:")
# display(pairs_df)